##ThinkDSP

This notebook contains solutions to exercises in Chapter 7: Discrete Fourier Transform

Copyright 2015 Allen Downey

License: [Creative Commons Attribution 4.0 International](http://creativecommons.org/licenses/by/4.0/)

In [3]:
from __future__ import print_function, division

import thinkdsp
import thinkplot
import thinkstats2
import math
import numpy as np

%precision 3
%matplotlib inline

PI2 = 2 * math.pi
i = complex(0, 1)

**Exercise:** In this chapter, I showed how we can express the DFT and inverse
DFT as matrix multiplications.  These operations are relatively
fast, taking time proportional to $N^2$, where $N$ is the length
of the wave array.  That would be fast enough for many applications,
but it turns out that there is a faster algorithm, the
Fast Fourier Transform (FFT), which takes time proportional to 
$N \log N$.

Read about the FFT at
http://en.wikipedia.org/wiki/Cooley-Tukey_FFT_algorithm, and
write an implementation.  Hint: I suggest you write a simple version
as a recursive function; don't worry about ``data reordering, bit
reversal, and in-place algorithms''.

As the test case, I'll start with a small real signal and compute its FFT:

In [4]:
ys = [-0.5, 0.1, 0.7, -0.1]
hs = np.fft.fft(ys)
print(hs)

[ 0.2+0.j  -1.2-0.2j  0.2+0.j  -1.2+0.2j]


Here's my implementation of DFT from the book:

In [5]:
def dft(ys):
    N = len(ys)
    ts = np.arange(N) / N
    freqs = np.arange(N)
    args = np.outer(ts, freqs)
    M = np.exp(i * PI2 * args)
    amps = M.conj().transpose().dot(ys)
    return amps

We can confirm that this implementation gets the same result.

In [6]:
hs2 = dft(ys)
print(sum(abs(hs - hs2)))

5.86477584677e-16


As a step toward making a recursive FFT, I'll start with a version that splits the input array and uses np.fft.fft to compute the FFT of the halves.

In [7]:
def fft(ys):
    N = len(ys)
    He = np.fft.fft(ys[::2])
    Ho = np.fft.fft(ys[1::2])
    
    ns = np.arange(N)
    W = np.exp(-i * PI2 * ns / N)
    
    return np.tile(He, 2) + W * np.tile(Ho, 2)

And we get the same results:

In [8]:
hs3 = fft(ys)
print(sum(abs(hs - hs3)))

0.0


Finally, we can replace `np.fft.fft` with recursive calls, and add a base case:

In [9]:
def fft(ys):
    N = len(ys)
    if N == 1:
        return ys
    
    He = fft(ys[::2])
    Ho = fft(ys[1::2])
    
    ns = np.arange(N)
    W = np.exp(-i * PI2 * ns / N)
    
    return np.tile(He, 2) + W * np.tile(Ho, 2)

And we get the same results:

In [10]:
hs4 = fft(ys)
print(sum(abs(hs - hs4)))

1.66533453694e-16


This implementation of FFT takes time proportional to $n \log n$.  It also takes space proportional to $n \log n$, and it wastes some time making and copying arrays.  It can be improved to run "in place", so it requires no additional space, and spends less time on overhead.